In [2]:
import os
import re
from nltk.tokenize import TreebankWordTokenizer
import pandas as pd

from nltk.stem import WordNetLemmatizer

In [3]:
ready_df = pd.DataFrame()

In [4]:
def preprocess(text):

    tokenized_text = TreebankWordTokenizer().tokenize(text)
    
    wordsFiltered = []

    for w in tokenized_text:
        for wf in re.findall(r'(?u)(\w+)', w):
            #for wfreal in re.findall(r'(\D+)', wf):
            wordsFiltered.append(''.join(wf))
    
    return wordsFiltered

In [5]:
def get_context(file_path):
    
    with open(file_path) as fl:
        try:
            text = fl.read()
        except:
            print ("Error: " + file_path )
    
    reg_term = re.compile(r'&(.*?)!&')
    indexes = []
    for i in reg_term.finditer(text):
        indexes.append((i.start(), i.end()))

    left_context = []
    right_context = []
    left_word = []

    for ind in indexes:
        left_part = text[:ind[0]]
        right_part = text[ind[1]+1:]
        clear_left = preprocess(left_part)
        clear_right = preprocess(right_part)
    
    
        #left context
        if len(clear_left) > 5:
            left_context.append(clear_left[-5:])
            left_word.append(clear_left[-1])
        else:
            left_context.append(clear_left)
            if len(clear_left) > 0:
                left_word.append(clear_left[-1])
            else:
                left_word.append(' ')
    
        #right context
        if len(clear_right) > 5:
            right_context.append(clear_right[:5])
        else:
            right_context.append(clear_right)
            
    
    terms = reg_term.findall(text)
    
    global ready_df
    lemmatizer = WordNetLemmatizer()
    
    
    for i in range(len(terms)):
        lft = left_context[i]
        lw = str(left_word[i])
        term = terms[i]
        rgt = right_context[i]
        lc = ' '.join(lft)
        rc = ' '.join(rgt)
        
        df = pd.DataFrame([[lc, lw, term, rc]],
                      columns=['left_context', 'left_word', 'term', 'right_context'])
        ready_df = ready_df.append(df)


In [6]:
start_path = './terms_markup/Marked/'

for path, dirs, files in os.walk(start_path):

    for fname in files:
        if not fname.startswith('.'):
            get_context(start_path+fname)


In [7]:
from rutermextract import TermExtractor

def get_term_candidates(text):
    
    terms = []
    term_extractor = TermExtractor()

    for term in term_extractor(text):
        terms.append((term.normalized, term.count))

In [8]:
df_sorted = ready_df.sort_values(['left_word'])

In [9]:
df_sorted.head()

,left_context,left_word,term,right_context
0,,,Маглев,на тестовом запуске в префектуре
0,,,Изменение возмущений в грунте,в зависимости от скорости снаряда
0,,,Облако,с точки зрения пользователя это
0,уроки соседа сверху каждые 10,10,децибел,громкость звука становится в два
0,в нашей на расстоянии 1350,1350,световых лет,от Земли в созвездии Ориона


In [10]:
ready_df.to_csv('terms_context.csv', sep='\t')

In [11]:
df_sorted.to_csv('terms_sorted.tsv', sep = '\t')

In [12]:
lemma = WordNetLemmatizer()

In [13]:
import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tag import tnt
from nltk.corpus import treebank
from nltk.corpus.reader.conll import *

In [14]:
from pymystem3 import Mystem

m = Mystem()

In [15]:
unigrams = []
bigrams = []
trigrams = []
multigrams = []

lem_unigrams = []
lem_bigrams = []
lem_trigrams = []
lem_multigrams = []

for term in df_sorted['term']:
    if len(term.split()) == 1:
        unigrams.append(term.lower())
        lem_unigrams.append(m.lemmatize(term))
    elif len(term.split()) == 2:
        bigrams.append(term.lower())
        lem_bigrams.append(m.lemmatize(term))
    elif len(term.split()) == 3:
        trigrams.append(term.lower())
        lem_trigrams.append(m.lemmatize(term))
    else:
        multigrams.append(term.lower())
        lem_multigrams.append(m.lemmatize(term))
        

In [16]:
with open('rus_bigram.txt') as fl:
    bi = fl.read()
    
with open('rus_trigram.txt') as fl:
    tri = fl.read()
    
with open('rus_tetragram.txt') as fl:
    tetra = fl.read()
    

In [17]:
bi_freq = bi.split('\n')
tri_freq = tri.split('\n')
tetra_freq = tetra.split('\n')

In [18]:
def text_to_dict(list_of_ngrams):
        
    unique_ngrams = list(set(list_of_ngrams))

    numbers = []
    for item in unique_ngrams:
        word_num = list_of_ngrams.count(item)
        numbers.append(word_num)
    
    dictionary = dict(zip(unique_ngrams, numbers))
    
    return dictionary

In [19]:
def get_ngrams(list_of_words, n):
    ngrams = []
    for word in list_of_words:
        for i in range(len(word)-n+1):
            ngrams.append(word[i:i+n].lower())

    for n in ngrams:
        if '-' in n.split():
            ngrams.remove(n)
    
    n_dict = text_to_dict(ngrams)
    
    return n_dict

In [20]:
with open('add_terms.txt') as fl:
    wiki = fl.read()

In [21]:
wiki_terms = wiki.split('\n')

In [22]:
final_unigrams = []

for u in lem_unigrams:
    final_unigrams.append(u[0])

In [23]:
terms = final_unigrams + wiki_terms

In [24]:
flat_list = ' '.join(terms)

In [25]:
term_bi = get_ngrams(terms, 2)

In [26]:
from collections import Counter

top_bi = []
for i in range(300):
    top_bi.append(Counter(term_bi).most_common(300)[i][0])

In [27]:
unique_bi = []
for i in top_bi:
    if i not in bi_freq:
        unique_bi.append(i)

In [28]:
print(unique_bi)

['фи', 'зи', 'нд', 'кс', 'ге', 'фо', 'фа', 'нг', 'нк', 'иф', 'рт', 'мп', 'ип', 'кц', 'пи', 'фр', 'эк', 'ев', 'аф', 'ба', 'еф', 'лл', 'рм', 'рф', 'уа', 'ум', 'кв', 'ук', 'и́', 'яц', 'рк', 'а́', 'ец', 'ау', 'фу', 'ке', 'фф', 'эм', 'мм', 'дж', 'пт', 'пп', 'мн', '́н', 'о́', 'нф', 'оч', 'эс', 'пс', 'нв', 'сц', 'рц', 'фл', 'гу', 'зе', 'е́', 'ца', 'эп', 'рв', 'хи', 'уи', 'уз', 'рп', '́т', 'кн', 'эл', 'рд', 'хр', 'йр', 'сф', 'пц', 'оо', 'эн', 'рб', 'йк', 'тк', '́р', 'еа', 'яр', 'кк', 'гм', 'уц', 'хе', 'ье', 'ха', 'см', 'вт', 'эр', 'юз', 'ао', 'мк', 'юм', 'йд', 'оэ', '́я', 'эг', 'co', 'кз', 'гг', 'йт', '́м', 'тт', '́з']


In [29]:
term_tri = get_ngrams(terms, 3)

top_tri = []
for i in range(300):
    top_tri.append(Counter(term_tri).most_common(300)[i][0])

In [30]:
print(top_tri)

['ция', 'аци', 'ный', 'ова', 'ост', 'ров', 'изм', 'иро', 'нос', 'сть', 'ват', 'ент', 'ать', 'льн', 'али', 'аль', 'оло', 'ика', 'тер', 'лиз', 'иза', 'тал', 'лог', 'про', 'тро', 'ант', 'кон', 'тор', 'ьны', 'сти', 'нци', 'ати', 'гия', 'ист', 'оги', 'стр', 'нти', 'ато', 'кат', 'тив', 'они', 'ата', 'енц', 'ари', 'ивн', 'тич', 'ест', 'тра', 'ком', 'сте', 'ици', 'кци', 'вар', 'ори', 'ван', 'ден', 'ано', 'нан', 'мет', 'ген', 'рац', 'ери', 'инг', 'ома', 'ина', 'акт', 'тик', 'ани', 'дис', 'еск', 'рен', 'ект', 'ром', 'ман', 'вер', 'ифи', 'ние', 'нтн', 'ива', 'ичн', 'вны', 'ера', 'роп', 'ени', 'три', 'пре', 'ото', 'ите', 'аст', 'рин', 'окс', 'иде', 'иал', 'ник', 'оти', 'зат', 'ски', 'иче', 'мен', 'тет', 'чес', 'пер', 'мат', 'био', 'ном', 'ион', 'ети', 'ези', 'зац', 'мин', 'иац', 'ста', 'ств', 'анс', 'род', 'тац', 'ета', 'оме', 'ита', 'пар', 'рив', 'цио', 'нац', 'кти', 'кац', 'рок', 'риз', 'ере', 'тиз', 'яци', 'ран', 'ляц', 'тво', 'ьно', 'диа', 'онд', 'иль', 'мия', 'фор', 'кий', 'экс', 'инт', 'сси'

In [31]:
print(tri_freq)

['рав', 'ств', 'пра', 'ени', 'ать', 'ове', 'ани', 'лов', 'сво', 'ело', 'чел', 'ост', 'ого', 'ния', 'ста', 'ест', 'век', 'аво', 'ажд', 'ние', 'льн', 'име', 'ова', 'при', 'каж', 'обо', 'или', 'жды', 'тат', 'воб', 'дый', 'бод', 'аци', 'его', 'еет', 'тья', 'мее', 'лен', 'тва', 'ных', 'енн', 'тво', 'аль', 'ван', 'сто', 'ьно', 'про', 'ног', 'ако', 'нов', 'сти', 'бра', 'пол', 'олж', 'дол', 'все', 'раз', 'ным', 'тор', 'нос', 'жен', 'ред', 'общ', 'ель', 'тел', 'оди', 'чес', 'как', 'сно', 'тве', 'има', 'пре', 'щес', 'ним', 'вле', 'нны', 'тьс', 'нно', 'шен', 'ься', 'аст', 'осу', 'быт', 'сов', 'ыть', 'осн', 'одн', 'лжн', 'нию', 'ран', 'бще', 'что', 'ват', 'оль', 'чен', 'сту', 'иче', 'рес', 'ном', 'ден', 'ежд', 'нац', 'вер', 'оже', 'рин', 'вен', 'туп', 'род', 'зов', 'нар', 'стр', 'обр', 'еоб', 'дос', 'есп', 'она', 'ото', 'ьны', 'лич', 'нии', 'тоя', 'аро', 'бес', 'гос', 'кот', 'суд', 'жно', 'ное', 'ава', 'под', 'печ', 'нст', 'уда', 'дно', 'ной', 'авн', 'ече', 'ход', 'нен', 'ции', 'уще', 'ате', 'льс'

In [32]:
unique_tri = []
for i in top_tri:
    if i not in tri_freq:
        unique_tri.append(i)

In [33]:
print(unique_tri)

['ция', 'ный', 'изм', 'иро', 'ент', 'али', 'оло', 'ика', 'лиз', 'иза', 'тал', 'лог', 'тро', 'ант', 'нци', 'ати', 'гия', 'ист', 'оги', 'нти', 'ато', 'кат', 'тив', 'они', 'ата', 'енц', 'ари', 'ивн', 'тич', 'ком', 'сте', 'ици', 'кци', 'вар', 'ори', 'нан', 'мет', 'ген', 'ери', 'инг', 'ома', 'акт', 'тик', 'дис', 'рен', 'ект', 'ифи', 'нтн', 'ива', 'ера', 'роп', 'три', 'окс', 'иде', 'оти', 'зат', 'ски', 'мен', 'тет', 'пер', 'мат', 'био', 'ети', 'ези', 'зац', 'мин', 'иац', 'анс', 'тац', 'ета', 'оме', 'ита', 'пар', 'рив', 'кти', 'кац', 'рок', 'риз', 'тиз', 'яци', 'ляц', 'диа', 'онд', 'иль', 'мия', 'фор', 'кий', 'экс', 'инт', 'сси', 'рны', 'тен', 'оми', 'цир', 'изи', 'зия', 'кол', 'фар', 'фил', 'тин', 'рия', 'оби', 'нте', 'етр', 'тно', 'нтр', 'тны', 'анд', 'гле', 'топ', 'ана', 'кси', 'нес', 'ерс', 'оно', 'нде', 'рон', 'дро', 'сия', 'фик', 'мон', 'лин', 'омб', 'ико', 'оли', 'зон', 'ане', 'риа', 'нта', 'офи', 'рма', 'лек', 'пли', 'бан', 'мед', 'уля', 'кса', 'мби', 'офо', 'ино', 'фер', 'оге', 'фра'

In [34]:
term_4 = get_ngrams(terms, 4)

top_4 = []
for i in range(300):
    top_4.append(Counter(term_4).most_common(300)[i][0])

In [35]:
unique_4 = []
for i in top_4:
    if i not in tetra_freq:
        unique_4.append(i)

In [36]:
print(unique_4)

['ация', 'иров', 'рова', 'ализ', 'олог', 'ьный', 'логи', 'енци', 'огия', 'лиза', 'атор', 'ката', 'кция', 'вный', 'ивны', 'тали', 'атал', 'тивн', 'изат', 'нция', 'зато', 'иаци', 'нано', 'таци', 'тика', 'ески', 'яция', 'ляци', 'каци', 'ский', 'рный', 'циро', 'тичн', 'дент', 'тный', 'анти', 'ентн', 'енти', 'нный', 'рокс', 'ризм', 'диац', 'ьнос', 'тери', 'рива', 'ерен', 'тром', 'меди', 'икац', 'тиче', 'арок', 'асти', 'исти', 'тнос', 'нтно', 'иоло', 'окса', 'лизм', 'комп', 'чнос', 'есте', 'ромб', 'нест', 'едиа', 'иден', 'варо', 'анес', 'ильн', 'итет', 'тези', 'ивар', 'троп', 'саба', 'тизм', 'ицир', 'чный', 'фика', 'зонд', 'ифик', 'метр', 'атив', 'нтро', 'абан', 'ксаб', 'мент', 'стез', 'тиро', 'мета', 'нтич', 'арфа', 'варф', 'ичны', 'фари', 'рфар', 'отоп', 'маци', 'арин', 'ропо', 'оген', 'оман', 'фици', 'итац', 'стиц', 'инте', 'рати', 'конс', 'офил', 'лект', 'спек', 'пара', 'низм', 'коти', 'ктив', 'атер', 'ндер', 'омби', 'топл', 'икот', 'стру', 'факт', 'мбин', 'трац', 'нтны', 'угле', 'фоби',

In [37]:
fl = open('tetragrams.txt', 'w')
for t in unique_4:
    fl.write(t + '\n')

fl.close()

In [38]:
fl = open('trigrams.txt', 'w')
for t in unique_tri:
    fl.write(t + '\n')

fl.close()

In [39]:
fl = open('bigrams.txt', 'w')
for t in unique_bi:
    fl.write(t + '\n')

fl.close()

In [40]:
df = pd.read_csv('./Freq2011/freqrnc2011.csv', sep='\t')
freq_dict = df[['Lemma','Freq(ipm)']]
sorted_freq = freq_dict.sort_values(by=['Freq(ipm)'])

In [41]:
lemmas = list(freq_dict['Lemma'])
values = list(freq_dict['Freq(ipm)'])

dictionary = dict(zip(lemmas, values))

In [42]:
freq_range = []

for term in terms:
    freq_range.append(dictionary.get(term))

In [43]:
freq_range = [x for x in freq_range if x != None]

In [44]:
print(freq_range)

[43.0, 0.59999999999999998, 0.5, 0.80000000000000004, 8.3000000000000007, 0.40000000000000002, 119.59999999999999, 0.5, 8.3000000000000007, 3.7999999999999998, 8.4000000000000004, 1.8999999999999999, 5.7000000000000002, 26.800000000000001, 4.0999999999999996, 1.7, 28.399999999999999, 14.300000000000001, 0.5, 8.3000000000000007, 6.2000000000000002, 2.1000000000000001, 6.2000000000000002, 0.5, 14.300000000000001, 7.7000000000000002, 1.0, 0.80000000000000004, 2.7000000000000002, 19.399999999999999, 0.69999999999999996, 1.5, 5.2000000000000002, 43.0, 5.7000000000000002, 57.5, 20.300000000000001, 1.7, 69.299999999999997, 1.1000000000000001, 1.1000000000000001, 6.2000000000000002, 0.80000000000000004, 0.40000000000000002, 2.2999999999999998, 37.100000000000001, 0.40000000000000002, 8.3000000000000007, 0.69999999999999996, 8.3000000000000007, 8.3000000000000007, 0.69999999999999996, 3.2999999999999998, 0.59999999999999998, 3.5, 3.7999999999999998, 10.4, 8.3000000000000007, 0.40000000000000002

In [45]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

l=[stemmer.stem(word) for word in terms]


In [46]:
suffixes3 = []
suffixes4 = []
prefixes3 = []
prefixes4 = []

for morph in l:
    suffixes3.append(morph[-3:])
    suffixes4.append(morph[-4:])
    prefixes3.append(morph[:3])
    prefixes4.append(morph[:4])
    

In [47]:
suf3 = list(set(suffixes3) - set(tri_freq))

In [48]:
suf4 = list(set(suffixes4) - set(tetra_freq))

In [49]:
pref3 = list(set(prefixes3) - set(tri_freq))

In [50]:
pref4 = list(set(prefixes4) - set(tetra_freq))

In [51]:
print(suf3)

['бан', 'лиз', 'лир', 'ечк', 'ета', 'бат', 'чик', 'сис', 'хул', 'прэ', 'лит', 'мик', 'арн', 'гез', 'ерз', 'ье́', 'арш', 'зол', 'теч', 'чек', 'фем', 'юзл', 'етр', 'ушк', 'пат', 'дра', 'туц', 'акц', 'гон', 'емн', 'риа', 'мин', 'ант', 'атн', 'оэп', 'орм', 'роп', 'си́', 'ндр', '-кв', 'нер', 'опе', 'нир', 'лак', 'пор', 'вра', 'бэк', 'арг', 'азн', 'ера', 'иац', 'айс', 'апс', 'тим', 'о́з', 'инг', 'аме', 'рбц', 'омн', 'тет', 'топ', 'арк', 'онг', 'зус', 'нез', 'нтн', 'ноф', 'куц', 'урн', 'пир', 'и́т', 'бин', 'ибр', 'амп', 'онк', 'ктр', 'отн', 'фоб', 'лев', 'пац', 'иум', 'ing', 'ill', 'таф', 'итн', 'тен', 'мот', 'ю́д', 'упп', 'епт', 'шиз', 'ри́', 'алл', 'над', 'кем', 'ига', 'абр', 'орп', 'льф', 'тес', 'инд', 'аор', 'лот', 'нсц', 'нкт', 'вац', 'тип', 'иат', 'ект', 'роб', 'фин', 'зюм', 'ерс', 'озн', 'аск', 'гиу', 'нет', 'уум', 'зи́', 'еоф', 'ажн', 'сH4', 'маз', 'инч', 'уац', 'люз', 'лет', 'тур', 'емм', 'SLS', 'мрт', 'сн4', 'амк', 'нат', 'iO2', 'а́л', 'лус', 'сан', 'нгв', 'ерк', 'азм', 'oop', 'пук'

In [52]:
fl = open('suf3.txt', 'w')
for s in suf3:
    if len(s) == 3:
        fl.write(s + '\n')
fl.close()

In [56]:
fl = open('suf4.txt', 'w')
for s in suf4:
    if len(s) == 4:
        fl.write(s + '\n')
fl.close()

In [54]:
fl = open('pref3.txt', 'w')
for s in pref3:
    if len(s) == 3:
        fl.write(s + '\n')
fl.close()

In [55]:
fl = open('pref4.txt', 'w')
for s in pref4:
    if len(s) == 4:
        fl.write(s + '\n')
fl.close()